<a href="https://colab.research.google.com/github/Daramanohar/alzheimers_detection/blob/main/alzheimers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Dependencies:

In [ ]:
!unzip -q "/content/alzheimers calssification.zip" -d /content/alzheimers_classification



In [ ]:
import os

# Define necessary directories
directories = [
    "/content/alzheimers_classification/alzheimers calssification/Alzheimer_Detection/data/train",
    "/content/alzheimers_classification/alzheimers calssification/Alzheimer_Detection/data/val",
    "/content/models",
    "/content/models/VGG16",
    "/content/models/VGG19",
    "/content/models/InceptionV3"
]

# Create directories if they do not exist
for directory in directories:
    os.makedirs(directory, exist_ok=True)

print("All required directories are created successfully!")


In [ ]:
!pip install numpy pandas matplotlib seaborn opencv-python scikit-learn tensorflow keras tqdm albumentations pillow
!pip install reportlab seaborn


data_preprocessing

In [ ]:
import os
import cv2
import numpy as np
import albumentations as A
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img

# ----- Step 1: Define Preprocessing Functions -----

# 1. Segmentation: (A simplistic example using Otsu's threshold)
def segment_brain(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Apply Otsu's thresholding to create a mask
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Use the mask to segment the image (assumes brain tissue appears brighter)
    segmented = cv2.bitwise_and(image, image, mask=thresh)
    return segmented

# 2. Skull Stripping: (Placeholder function; for real cases, consider specialized neuroimaging tools)
def skull_strip(image):
    # For demonstration, assume segmentation did most of the work.
    # You could apply further morphological operations here.
    return image

# 3. Spatial Normalization & Re-scaling: (Resize image to a standard target size)
def spatial_normalize(image, target_size=(224, 224)):
    normalized = cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)
    return normalized

# 4. Data Augmentation: (Using Albumentations)
augmentation_pipeline = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.5)
])

# 5. Smoothing: (Apply Gaussian Blur)
def smooth_image(image):
    smoothed = cv2.GaussianBlur(image, (5, 5), 0)
    return smoothed

# ----- Step 2: Full Preprocessing Pipeline -----

def preprocess_image(image_path, target_size=(224, 224)):
    # Load image using Keras (PIL-based) and convert to a NumPy array
    img = load_img(image_path)
    image = img_to_array(img).astype(np.uint8)

    # Apply segmentation to isolate the brain region
    segmented = segment_brain(image)

    # Apply skull stripping (placeholder function)
    stripped = skull_strip(segmented)

    # Spatially normalize & re-scale the image to the target size
    normalized = spatial_normalize(stripped, target_size=target_size)

    # Data augmentation: apply random flips/rotations/scaling
    augmented = augmentation_pipeline(image=normalized)['image']

    # Smoothing: reduce noise with Gaussian blur
    smoothed = smooth_image(augmented)

    # Final normalization: scale pixel values to [0, 1]
    processed = smoothed / 255.0
    return processed

# ----- Step 3: Process Entire Folder Structure -----

# Define base input and output directories (update paths as needed)
input_base = r'/content/alzheimers_classification/alzheimers calssification/Alzheimer_Detection/data/raw/Axial'
output_base = r'/content/alzheimers_classification/alzheimers calssification/Alzheimer_Detection/data/processed'
categories = ['AD', 'CMI', 'CN']

for category in categories:
    input_dir = os.path.join(input_base, category)
    output_dir = os.path.join(output_base, category)

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Process each image in the category folder
    for img_name in os.listdir(input_dir):
        img_path = os.path.join(input_dir, img_name)
        try:
            processed_img = preprocess_image(img_path, target_size=(224, 224))
            # Convert the processed image (which is normalized) back to uint8 for saving
            processed_img_uint8 = (processed_img * 255).astype(np.uint8)
            output_path = os.path.join(output_dir, img_name)
            save_img(output_path, processed_img_uint8)
            print(f"Processed and saved: {img_path} --> {output_path}")
        except Exception as e:
            print(f"Error processing {img_path}: {e}")


split your preprocessed images into training and validation

In [ ]:
import os
import shutil
import random

# Set the seed for reproducibility
random.seed(42)

# Define the source folder where your preprocessed images are stored
source_dir = '/content/alzheimers_classification/alzheimers calssification/Alzheimer_Detection/data/processed'

# Define destination base directory for train and validation sets
dest_base = '/content/alzheimers_classification/alzheimers calssification/Alzheimer_Detection/data'
train_dir = os.path.join(dest_base, 'train')
val_dir = os.path.join(dest_base, 'val')

# Define the split ratio (e.g., 80% training, 20% validation)
split_ratio = 0.8

# Create the destination directories if they don't exist
for directory in [train_dir, val_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Process each category (AD, MCI, NC)
categories = ['AD', 'CMI', 'CN']
for category in categories:
    source_category = os.path.join(source_dir, category)

    # Create category subdirectories for train and val
    train_category = os.path.join(train_dir, category)
    val_category = os.path.join(val_dir, category)
    os.makedirs(train_category, exist_ok=True)
    os.makedirs(val_category, exist_ok=True)

    # Get list of images/files for this category and filter out directories
    images = [img for img in os.listdir(source_category) if os.path.isfile(os.path.join(source_category, img))]
    random.shuffle(images)

    # Calculate split index
    split_index = int(len(images) * split_ratio)
    train_images = images[:split_index]
    val_images = images[split_index:]

    # Copy images to train folder
    for img in train_images:
        src = os.path.join(source_category, img)
        dst = os.path.join(train_category, img)
        shutil.copy(src, dst)

    # Copy images to validation folder
    for img in val_images:
        src = os.path.join(source_category, img)
        dst = os.path.join(val_category, img)
        shutil.copy(src, dst)

    print(f"Category '{category}': {len(train_images)} images in train, {len(val_images)} in val.")


Model Implementation Using Transfer Learning VGG-16

In [ ]:
import os
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16  # Switching back to VGG16 which worked better for you
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
from datetime import datetime
import json

# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Hyperparameters - optimized for VGG16
IMG_SIZE = (224, 224)  # Standard size for VGG16
BATCH_SIZE = 32  # Balanced for training stability
EPOCHS = 50  # More epochs with early stopping
NUM_CLASSES = 3  # AD, MCI, CN
INITIAL_LR = 1e-4  # Lower initial learning rate for more stable training
WEIGHT_DECAY = 1e-4  # Increased L2 regularization

# Data directories (update these paths to your environment)
train_dir = '/content/alzheimers_classification/alzheimers calssification/Alzheimer_Detection/data/train'
val_dir = '/content/alzheimers_classification/alzheimers calssification/Alzheimer_Detection/data/val'

# Create output directories if they don't exist
model_dir = 'Alzheimer_Detection/models'
os.makedirs(model_dir, exist_ok=True)
log_dir = os.path.join(model_dir, "logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
os.makedirs(log_dir, exist_ok=True)

# Improved data augmentation strategy - but not too aggressive
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,           # Reduced rotation for brain scans
    zoom_range=0.15,             # Moderate zoom
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.1,             # Reduced shear (medical images)
    horizontal_flip=True,        # Flipping can be valid for brain scans
    vertical_flip=False,         # Brain scans typically have orientation significance
    fill_mode='nearest',         # Changed to nearest for medical images
    brightness_range=[0.85, 1.15], # Reduced brightness variation
    validation_split=0.2         # Using validation split for consistency
)

# Validation data with minimal processing
val_datagen = ImageDataGenerator(
    rescale=1./255
)

# Data generators with class weights
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=seed
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=seed
)

# Calculate class weights to handle potential class imbalance
class_weights_dict = None
try:
    # Get class indices
    classes = list(train_generator.class_indices.keys())
    # Count samples per class
    class_counts = [0] * len(classes)
    for i in range(len(train_generator.classes)):
        class_counts[train_generator.classes[i]] += 1

    # Calculate class weights
    total = sum(class_counts)
    class_weights = [total / (len(class_counts) * count) for count in class_counts]
    class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
    print("Class weights:", class_weights_dict)
except Exception as e:
    print(f"Could not compute class weights: {e}")

# Load VGG16 with pre-trained ImageNet weights
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_tensor=Input(shape=(*IMG_SIZE, 3))
)

# Freeze initial layers - only train the deeper layers for feature extraction
# For VGG16, freezing first 15 layers (out of 19) works well
for layer in base_model.layers[:15]:
    layer.trainable = False

# Build enhanced classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global pooling instead of flatten for better generalization

# First dense block
x = Dense(1024, kernel_regularizer=l2(WEIGHT_DECAY))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)  # ReLU works better with VGG16
x = Dropout(0.5)(x)  # Increased dropout for better regularization

# Second dense block
x = Dense(512, kernel_regularizer=l2(WEIGHT_DECAY))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = Dropout(0.5)(x)

# Output layer
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Use a fixed learning rate for training
optimizer = Adam(learning_rate=INITIAL_LR)

# Compile with gradient clipping to prevent exploding gradients
optimizer = tf.keras.optimizers.Adam(
    learning_rate=INITIAL_LR,
    clipnorm=1.0  # Add gradient clipping
)

# Compile model
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05),  # Reduced label smoothing
    metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

model.summary()

# Improved callbacks
early_stopping = EarlyStopping(
    monitor='val_accuracy',  # Monitor accuracy instead of loss
    patience=10,             # More patience
    restore_best_weights=True,
    verbose=1,
    min_delta=0.005          # Significant improvement
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',  # Monitor accuracy instead of loss
    factor=0.2,              # More aggressive reduction
    patience=5,
    min_lr=1e-7,
    verbose=1
)

checkpoint = ModelCheckpoint(
    os.path.join(model_dir, 'vgg16_best.keras'),  # Using new .keras format
    monitor='val_accuracy',  # Monitor accuracy instead of loss
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

# Add TensorBoard for visualization
tensorboard = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    update_freq='epoch'
)

# Initial training with semi-frozen base model
history = model.fit(
    train_generator,
    steps_per_epoch=math.ceil(train_generator.samples / BATCH_SIZE),
    validation_data=val_generator,
    validation_steps=math.ceil(val_generator.samples / BATCH_SIZE),
    epochs=25,  # Longer initial phase
    callbacks=[early_stopping, reduce_lr, checkpoint, tensorboard],
    class_weight=class_weights_dict,
    verbose=1
)

# Fine-tuning phase - unfreeze more layers
print("Starting fine-tuning phase...")

# Unfreeze more layers for fine-tuning
for layer in base_model.layers[10:]:  # Unfreeze the last few layers
    layer.trainable = True

# Use a lower learning rate for fine-tuning
ft_optimizer = Adam(learning_rate=5e-6)  # Much lower learning rate

# Recompile with lower learning rate
model.compile(
    optimizer=ft_optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Modify callbacks for fine-tuning phase
ft_early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=15,             # More patience for fine-tuning
    restore_best_weights=True,
    verbose=1,
    min_delta=0.001         # Smaller improvements matter now
)

ft_reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,              # Less aggressive reduction
    patience=7,
    min_lr=1e-8,
    verbose=1
)

ft_checkpoint = ModelCheckpoint(
    os.path.join(model_dir, 'vgg16_ft_best.keras'),
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

# Continue with fine-tuning
current_epoch = len(history.history['loss'])
ft_history = model.fit(
    train_generator,
    steps_per_epoch=math.ceil(train_generator.samples / BATCH_SIZE),
    validation_data=val_generator,
    validation_steps=math.ceil(val_generator.samples / BATCH_SIZE),
    epochs=current_epoch + 25,  # Additional 25 epochs
    initial_epoch=current_epoch,
    callbacks=[ft_early_stopping, ft_reduce_lr, ft_checkpoint, tensorboard],
    class_weight=class_weights_dict,
    verbose=1
)

# Save the final model
model.save(os.path.join(model_dir, 'vgg16_final_model.keras'))

# Combine histories
all_history = {}
for key in history.history:
    all_history[key] = history.history[key].copy()
    if key in ft_history.history:
        all_history[key].extend(ft_history.history[key])

# Wrap combined history
class HistoryWrapper:
    def __init__(self, history_dict):
        self.history = history_dict

# Function to visualize training history
def plot_training_history(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.savefig(os.path.join(model_dir, 'training_history.png'))
    plt.show()

# Visualize the training process
try:
    combined_history = HistoryWrapper(all_history)
    plot_training_history(combined_history)
except Exception as e:
    print(f"Could not plot training history: {e}")

# Evaluate the final model
final_results = model.evaluate(val_generator)
print(f"Final validation results: {dict(zip(model.metrics_names, final_results))}")

# Plot confusion matrix to better understand model performance
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Get predictions
val_generator.reset()
y_pred = model.predict(val_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = val_generator.classes

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(val_generator.class_indices.keys()),
            yticklabels=list(val_generator.class_indices.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig(os.path.join(model_dir, 'confusion_matrix.png'))
plt.show()

# Generate classification report
report = classification_report(y_true, y_pred_classes,
                              target_names=list(val_generator.class_indices.keys()),
                              output_dict=True)

# Print classification report
print("Classification Report:")
print(classification_report(y_true, y_pred_classes,
                          target_names=list(val_generator.class_indices.keys())))



VGG-19

In [ ]:
import os
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG19  # Switching back to VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
from datetime import datetime

# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Hyperparameters - optimized for VGG19
IMG_SIZE = (224, 224)  # Standard size for VGG19
BATCH_SIZE = 32  # Balanced for training stability
EPOCHS = 50  # More epochs with early stopping
NUM_CLASSES = 3  # AD, MCI, CN
INITIAL_LR = 1e-4  # Lower initial learning rate for more stable training
WEIGHT_DECAY = 1e-4  # Increased L2 regularization

# Data directories (update these paths to your environment)
train_dir = '/content/alzheimers_classification/alzheimers calssification/Alzheimer_Detection/data/train'
val_dir = '/content/alzheimers_classification/alzheimers calssification/Alzheimer_Detection/data/val'

# Create output directories if they don't exist
model_dir = 'Alzheimer_Detection/models'
os.makedirs(model_dir, exist_ok=True)
log_dir = os.path.join(model_dir, "logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
os.makedirs(log_dir, exist_ok=True)

# Improved data augmentation strategy - but not too aggressive
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,           # Reduced rotation for brain scans
    zoom_range=0.15,             # Moderate zoom
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.1,             # Reduced shear (medical images)
    horizontal_flip=True,        # Flipping can be valid for brain scans
    vertical_flip=False,         # Brain scans typically have orientation significance
    fill_mode='nearest',         # Changed to nearest for medical images
    brightness_range=[0.85, 1.15], # Reduced brightness variation
    validation_split=0.2         # Using validation split for consistency
)

# Validation data with minimal processing
val_datagen = ImageDataGenerator(
    rescale=1./255
)

# Data generators with class weights
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=seed
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=seed
)

# Calculate class weights to handle potential class imbalance
class_weights_dict = None
try:
    # Get class indices
    classes = list(train_generator.class_indices.keys())
    # Count samples per class
    class_counts = [0] * len(classes)
    for i in range(len(train_generator.classes)):
        class_counts[train_generator.classes[i]] += 1

    # Calculate class weights
    total = sum(class_counts)
    class_weights = [total / (len(class_counts) * count) for count in class_counts]
    class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
    print("Class weights:", class_weights_dict)
except Exception as e:
    print(f"Could not compute class weights: {e}")

# Load VGG19 with pre-trained ImageNet weights
base_model = VGG19(
    weights='imagenet',
    include_top=False,
    input_tensor=Input(shape=(*IMG_SIZE, 3))
)

# Freeze initial layers - adjusted for VGG19's structure (19 layers total)
for layer in base_model.layers[:17]:  # Freeze through block4_pool
    layer.trainable = False

# Build enhanced classification head (same as VGG16 version)
x = base_model.output
x = GlobalAveragePooling2D()(x)

# First dense block
x = Dense(1024, kernel_regularizer=l2(WEIGHT_DECAY))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = Dropout(0.5)(x)

# Second dense block
x = Dense(512, kernel_regularizer=l2(WEIGHT_DECAY))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = Dropout(0.5)(x)

# Output layer
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Optimizer setup (identical to VGG16 version)
optimizer = tf.keras.optimizers.Adam(
    learning_rate=INITIAL_LR,
    clipnorm=1.0
)

# Compile model (identical to VGG16 version)
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=['accuracy', tf.keras.metrics.AUC(),
            tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

model.summary()

# Improved callbacks
early_stopping = EarlyStopping(
    monitor='val_accuracy',  # Monitor accuracy instead of loss
    patience=10,             # More patience
    restore_best_weights=True,
    verbose=1,
    min_delta=0.005          # Significant improvement
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',  # Monitor accuracy instead of loss
    factor=0.2,              # More aggressive reduction
    patience=5,
    min_lr=1e-7,
    verbose=1
)

checkpoint = ModelCheckpoint(
    os.path.join(model_dir, 'vgg16_best.keras'),  # Using new .keras format
    monitor='val_accuracy',  # Monitor accuracy instead of loss
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

# Add TensorBoard for visualization
tensorboard = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    update_freq='epoch'
)

# Initial training with semi-frozen base model
history = model.fit(
    train_generator,
    steps_per_epoch=math.ceil(train_generator.samples / BATCH_SIZE),
    validation_data=val_generator,
    validation_steps=math.ceil(val_generator.samples / BATCH_SIZE),
    epochs=25,  # Longer initial phase
    callbacks=[early_stopping, reduce_lr, checkpoint, tensorboard],
    class_weight=class_weights_dict,
    verbose=1
)

# Fine-tuning phase - unfreeze more layers
print("Starting fine-tuning phase...")

# Unfreeze more layers for fine-tuning
for layer in base_model.layers[10:]:  # Unfreeze the last few layers
    layer.trainable = True

# Use a lower learning rate for fine-tuning
ft_optimizer = Adam(learning_rate=5e-6)  # Much lower learning rate

# Recompile with lower learning rate
model.compile(
    optimizer=ft_optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Modify callbacks for fine-tuning phase
ft_early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=15,             # More patience for fine-tuning
    restore_best_weights=True,
    verbose=1,
    min_delta=0.001         # Smaller improvements matter now
)

ft_reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,              # Less aggressive reduction
    patience=7,
    min_lr=1e-8,
    verbose=1
)

ft_checkpoint = ModelCheckpoint(
    os.path.join(model_dir, 'vgg16_ft_best.keras'),
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

# Continue with fine-tuning
current_epoch = len(history.history['loss'])
ft_history = model.fit(
    train_generator,
    steps_per_epoch=math.ceil(train_generator.samples / BATCH_SIZE),
    validation_data=val_generator,
    validation_steps=math.ceil(val_generator.samples / BATCH_SIZE),
    epochs=current_epoch + 25,  # Additional 25 epochs
    initial_epoch=current_epoch,
    callbacks=[ft_early_stopping, ft_reduce_lr, ft_checkpoint, tensorboard],
    class_weight=class_weights_dict,
    verbose=1
)

# Save the final model
model.save(os.path.join(model_dir, 'vgg19_final_model.keras'))

# Combine histories
all_history = {}
for key in history.history:
    all_history[key] = history.history[key].copy()
    if key in ft_history.history:
        all_history[key].extend(ft_history.history[key])

# Wrap combined history
class HistoryWrapper:
    def __init__(self, history_dict):
        self.history = history_dict

# Function to visualize training history
def plot_training_history(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.savefig(os.path.join(model_dir, 'training_history.png'))
    plt.show()

# Visualize the training process
try:
    combined_history = HistoryWrapper(all_history)
    plot_training_history(combined_history)
except Exception as e:
    print(f"Could not plot training history: {e}")

# Evaluate the final model
final_results = model.evaluate(val_generator)
print(f"Final validation results: {dict(zip(model.metrics_names, final_results))}")

# Plot confusion matrix to better understand model performance
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Get predictions
val_generator.reset()
y_pred = model.predict(val_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = val_generator.classes

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(val_generator.class_indices.keys()),
            yticklabels=list(val_generator.class_indices.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig(os.path.join(model_dir, 'confusion_matrix.png'))
plt.show()

# Generate classification report
report = classification_report(y_true, y_pred_classes,
                              target_names=list(val_generator.class_indices.keys()),
                              output_dict=True)

# Print classification report
print("Classification Report:")
print(classification_report(y_true, y_pred_classes,
                          target_names=list(val_generator.class_indices.keys())))



INCEPTION -V3


In [ ]:
import os
import math
import numpy as np
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
from datetime import datetime

# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Hyperparameters - optimized for VGG19
IMG_SIZE = (299, 299)   # Updated input size
BATCH_SIZE = 32  # Balanced for training stability
EPOCHS = 50  # More epochs with early stopping
NUM_CLASSES = 3  # AD, MCI, CN
INITIAL_LR = 1e-4  # Lower initial learning rate for more stable training
WEIGHT_DECAY = 1e-4  # Increased L2 regularization

# Data directories (update these paths to your environment)
train_dir = '/content/alzheimers_classification/alzheimers calssification/Alzheimer_Detection/data/train'
val_dir = '/content/alzheimers_classification/alzheimers calssification/Alzheimer_Detection/data/val'

# Create output directories if they don't exist
model_dir = 'Alzheimer_Detection/models'
os.makedirs(model_dir, exist_ok=True)
log_dir = os.path.join(model_dir, "logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
os.makedirs(log_dir, exist_ok=True)


# Use Inception-specific preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Scales pixels between -1 and 1
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)


# Data generators with class weights
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=seed
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=seed
)

# Calculate class weights to handle potential class imbalance
class_weights_dict = None
try:
    # Get class indices
    classes = list(train_generator.class_indices.keys())
    # Count samples per class
    class_counts = [0] * len(classes)
    for i in range(len(train_generator.classes)):
        class_counts[train_generator.classes[i]] += 1

    # Calculate class weights
    total = sum(class_counts)
    class_weights = [total / (len(class_counts) * count) for count in class_counts]
    class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
    print("Class weights:", class_weights_dict)
except Exception as e:
    print(f"Could not compute class weights: {e}")

# Load InceptionV3 base model
base_model = InceptionV3(weights='imagenet',
                        include_top=False,
                        input_shape=(*IMG_SIZE, 3))


for layer in base_model.layers[:249]:
    layer.trainable = False
for layer in base_model.layers[249:]:
    layer.trainable = True

# Add custom head (same structure)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)

predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Optimizer setup (identical to VGG16 version)
optimizer = tf.keras.optimizers.Adam(
    learning_rate=INITIAL_LR,
    clipnorm=1.0
)

model.compile(optimizer=Adam(learning_rate=1e-5, clipnorm=1.0),
            loss='categorical_crossentropy',
            metrics=['accuracy'])

model.summary()

# Improved callbacks
early_stopping = EarlyStopping(
    monitor='val_accuracy',  # Monitor accuracy instead of loss
    patience=10,             # More patience
    restore_best_weights=True,
    verbose=1,
    min_delta=0.005          # Significant improvement
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',  # Monitor accuracy instead of loss
    factor=0.2,              # More aggressive reduction
    patience=5,
    min_lr=1e-7,
    verbose=1
)

checkpoint = ModelCheckpoint(
    os.path.join(model_dir, 'vgg16_best.keras'),  # Using new .keras format
    monitor='val_accuracy',  # Monitor accuracy instead of loss
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

# Add TensorBoard for visualization
tensorboard = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    update_freq='epoch'
)

# Initial training with semi-frozen base model
history = model.fit(
    train_generator,
    steps_per_epoch=math.ceil(train_generator.samples / BATCH_SIZE),
    validation_data=val_generator,
    validation_steps=math.ceil(val_generator.samples / BATCH_SIZE),
    epochs=25,  # Longer initial phase
    callbacks=[early_stopping, reduce_lr, checkpoint, tensorboard],
    class_weight=class_weights_dict,
    verbose=1
)

# Fine-tuning phase - unfreeze more layers
print("Starting fine-tuning phase...")

# Unfreeze more layers for fine-tuning
for layer in base_model.layers[10:]:  # Unfreeze the last few layers
    layer.trainable = True

# Use a lower learning rate for fine-tuning
ft_optimizer = Adam(learning_rate=5e-6)  # Much lower learning rate

# Recompile with lower learning rate
model.compile(
    optimizer=ft_optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Modify callbacks for fine-tuning phase
ft_early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=15,             # More patience for fine-tuning
    restore_best_weights=True,
    verbose=1,
    min_delta=0.001         # Smaller improvements matter now
)

ft_reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,              # Less aggressive reduction
    patience=7,
    min_lr=1e-8,
    verbose=1
)

ft_checkpoint = ModelCheckpoint(
    os.path.join(model_dir, 'vgg16_ft_best.keras'),
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

# Continue with fine-tuning
current_epoch = len(history.history['loss'])
ft_history = model.fit(
    train_generator,
    steps_per_epoch=math.ceil(train_generator.samples / BATCH_SIZE),
    validation_data=val_generator,
    validation_steps=math.ceil(val_generator.samples / BATCH_SIZE),
    epochs=current_epoch + 25,  # Additional 25 epochs
    initial_epoch=current_epoch,
    callbacks=[ft_early_stopping, ft_reduce_lr, ft_checkpoint, tensorboard],
    class_weight=class_weights_dict,
    verbose=1
)

# Save the final model
model.save(os.path.join(model_dir, 'vgg19_final_model.keras'))

# Combine histories
all_history = {}
for key in history.history:
    all_history[key] = history.history[key].copy()
    if key in ft_history.history:
        all_history[key].extend(ft_history.history[key])

# Wrap combined history
class HistoryWrapper:
    def __init__(self, history_dict):
        self.history = history_dict

# Function to visualize training history
def plot_training_history(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.savefig(os.path.join(model_dir, 'training_history.png'))
    plt.show()

# Visualize the training process
try:
    combined_history = HistoryWrapper(all_history)
    plot_training_history(combined_history)
except Exception as e:
    print(f"Could not plot training history: {e}")

# Evaluate the final model
final_results = model.evaluate(val_generator)
print(f"Final validation results: {dict(zip(model.metrics_names, final_results))}")

# Plot confusion matrix to better understand model performance
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Get predictions
val_generator.reset()
y_pred = model.predict(val_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = val_generator.classes

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(val_generator.class_indices.keys()),
            yticklabels=list(val_generator.class_indices.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig(os.path.join(model_dir, 'confusion_matrix.png'))
plt.show()

# Generate classification report
report = classification_report(y_true, y_pred_classes,
                              target_names=list(val_generator.class_indices.keys()),
                              output_dict=True)

# Print classification report
print("Classification Report:")
print(classification_report(y_true, y_pred_classes,
                          target_names=list(val_generator.class_indices.keys())))

